In [0]:
%run ./00_config

In [0]:
spark.sql(f"""
-- 手書きボリュームのファイルの文字を抽出し、bronzeテーブルを作成
CREATE OR REPLACE TABLE {MY_CATALOG}.{MY_SCHEMA}.temp_bronze AS
SELECT
  path,
  ai_parse_document(content) AS content
FROM
  READ_FILES(
    '/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/image',
    format => 'binaryFile'
  );
""")

In [0]:
prompt = (
    '貴方はテキスト整理のプロフェッショナルです。温度チェック管理表についてJSON形式でまとめてください。'
    'parse_jsonできるように余計な文字列は入れないでください。日付、温度をまとめて一つのテキストに整形してください。'
    'Keyはそれぞれ"日付"、"温度"でお願いします。[で始まり、]で終わるJSONで返してください。```json```という文字列は抜いてください。'
)

spark.sql(f"""
-- bronzeテーブルから必要項目を切り出して、Silverをテーブルを作成
CREATE OR REPLACE TABLE {MY_CATALOG}.{MY_SCHEMA}.temp_silver AS
SELECT
  path,
  ai_query(
    'databricks-claude-sonnet-4',
    '{prompt}' || content
  ) AS summary
FROM
  {MY_CATALOG}.{MY_SCHEMA}.temp_bronze
""")

In [0]:
spark.sql(f"""
-- json形式に格納されたSilverテーブルの日付、温度の列を展開して、Goldテーブルを作成
CREATE OR REPLACE TABLE {MY_CATALOG}.{MY_SCHEMA}.temp_gold AS
SELECT
  TO_DATE(GET_JSON_OBJECT(case_element, '$.日付'), 'yyyy/M/d') AS checkdate, -- 日付を日付型に変換（正しいフォーマット指定）
  CAST(GET_JSON_OBJECT(case_element, '$.温度') AS DOUBLE) AS temperature -- 温度を数値型に変換
FROM
  {MY_CATALOG}.{MY_SCHEMA}.temp_silver
LATERAL VIEW
  EXPLODE(from_json(summary, 'ARRAY<STRING>')) exploded_cases AS case_element; -- cases配列を展開
""")